# Metage2Metabo

Metage2Metabo (m2m) is a python package to perform graph-based metabolic analysis starting from annotated genomes or metabolic networks.

This jupyter notebook uses the following versions of these dependencies:

- Metage2Metabo==1.5.0
- mpwt==0.6.0
- padmet==5.0.1
- MeneTools==3.1.1
- Miscoto==3.1.1
- bubbletools==0.6.11
- networkx==2.5

## `m2m iscope`: Individual scope

This part of the workflow encompasses the computation of the individual scope.

### Scope formalisation

We define a metabolic network as a bipartite graph $G = (R \cup M, E)$, where $R$ and $M$ stand for reaction and metabolite nodes.  When $(m,r) \in E$ (respectively $(r,m) \in E$), with $m \in M$  and $r \in R$, the metabolite is called a _reactant_ (respectively _product_) of the reaction $r$. The scope of a set of seed compounds $S$ according to a metabolic network $G$, denoted by $Scope(G, S)$, is iteratively computed until it reaches a fixed point (Handorf and Ebenhöh 2005). It is formally defined by:

$$\Scope(G, S) = \bigcup_iM_i, \mbox{ where } M_0 = S \mbox{ and }
M_{i+1} = M_i \cup \Products{\{ r \in R \mid \Reactants\{r\} \subseteq M_i\}}.$$

### An example

In the following gif, we look at a metabolic network with 3 reactions (R1, R2, R3) and 6 metabolites (A, B, C, D, E, F).

The seeds are composed of metabolites A and B.

With these inputs, the scope will be compute. As A and B are available as seed, they can be used as reactant to activate the reaction R1, which will produce metabolite D. But the reaction R2 will not be activated as the metabolite C is not availabe in the seeds.

The metabolite D, produced by the reaction R1, can activate the reaction R3 to produce the metabolite F.

<p>
    <img src='images/iscope.gif' width='30%'>
    <center><em>Figue 1: Individual scope</em></center>
</p>

### Tutorial

For this tutorial, we will use this example:

In [2]:
# Instanciate the logger
import logging

logging.basicConfig()

# Run the individual scope with metage2metabo on the folder of SBML using the seeds files.
# This is similar to the command:
# m2m iscope -n data/community -s data/seeds.sbml -o output_folder
from metage2metabo.m2m import individual_scope

individual_scope.iscope('data/community', 'data/seeds.sbml', 'output_folder')


{'M_A_c', 'M_B_c', 'M_D_c', 'M_F_c'}

In [3]:
# Read the result

import json

with open('output_folder/indiv_scopes/indiv_scopes.json') as iscope_output:
    iscope_json_data = json.load(iscope_output)

print('Individual scope:')

print('{0}: {1} metabolites ({2}).'.format('OrgA', len(iscope_json_data['OrgA']), ', '.join(sorted(iscope_json_data['OrgA']))))

Individual scope:
OrgA: 4 metabolites (M_A_c, M_B_c, M_D_c, M_F_c).


## `m2m cscope`: Community scope

This part encompasses the computation of the community scope.

### Community scope formalisation

The m2m cscope command computes the metabolic capabilities of the whole microbiota by taking into account the complementarity of metabolic pathways between metabolic networks. This step simulates the sharing of metabolic biosynthesis through a meta-organism composed of all metabolic networks, and assesses the metabolic compounds that can be reached using network expansion. This calculation is an extension of the features of [MiSCoTo](https://github.com/cfrioux/miscoto) in which the collective scope of a collection of metabolic networks $\{G_1, \dots G_N\}$ is introduced. We define:

$$CommunityScope(G_1.. G_N, S)=Scope \left( \left(\bigcup_{i \in {\{1..n\}}} R_i, \bigcup_{i \in {\{1..n\}}} M_i, \bigcup_{i \in {\{1..n\}}} E_i \right), S \right).$$ 

### An example

In this example, we have 3 metabolic networks (OrgA, OrgB and OrgC). OrgA has 3 reactions (R1, R2 and R3). OrgB and OrgC have each 2 reactions, one they shared (R5) and one specific to each (R4 an R6).
The formalism behind the community scope of m2m uses a Mixed Bag modelling, which considers the metabolic networks of the community as a meta-organism allowing exchanges between them without cost (described in the figure as a dotted line). 

As for the Individual Scope, the seeds are composed of two metabolites (A and B).

With these inputs, the Community Scope can be computed. As A and B are available, the reaction R1 is activated, producing the metabolite D. But reaction R2 is not activated as one of the reactant (metabolite C) is not available. Then with D metabolite producible, the reaction R3 can be activated and produces metabolite F.

No reaction in the metabolic network OrgA takes the metabolite F as reactant but the reaction R5 of metabolic networks OrgB and OrgC does. With the Mixed Bag modelling, the metabolite F is then available to be used by these organisms. This activates both R5 reactions and produces H. The reactions R4 and R6 can not be activated as their reactants are not available.

The Community Scope of OrgA, OrgB and OrgC is composed of the metabolites D, F and H.

<p>
    <img src='images/comscope.gif' width='60%'>
    <center><em>Figue 2: Community scope</em></center>
</p>

In [4]:
# Run the community scope with metage2metabo on the folder of SBML using the seeds files.
# This is similar to the command:
#m2m cscope -n data/community -s data/seeds.sbml -o output_folder
from metage2metabo.m2m import community_scope

instance_path, network_scopes = community_scope.cscope('data/community', 'data/seeds.sbml', 'output_folder')

print('The community scope contains {0} metabolites: {1}.'.format(len(network_scopes), ', '.join(sorted(network_scopes))))

The community scope contains 3 metabolites: M_D_c, M_F_c, M_H_c.


## `m2m addedvalue`: Cooperation potential

This part encompasses the computation of the cooperation potential.

### Cooperation potential formalisation

Given individual and community metabolic potentials, the _cooperation potential_ consists in the set of metabolites whose producibility can only occur if several organisms participate in the biosynthesis. `m2m addedvalue` computes the cooperation potential by performing a set difference between the community scope and the union of individual scopes, and produces a SBML file with the resulting metabolites. This list of compounds is inclusive and could comprise false positives not necessitating cooperation for production, but selected due to missing annotations in the initial genomes. One can modify the SBML file accordingly, prior to the following M2M community reduction step.

The cooperation potential $\coopPot(G_1,..,G_n,S)$ of a collection of metabolic networks $\{G_1..G_n\}$ is defined by:


$\coopPot(G_1,..,G_n,S) = \mixedbagScope(G_1,..,G_n,S)  \setminus  \bigcup_{i \in {\{1..n\}}} \Scope(G_i,S).$

### An example

In the **Figure 2**, the cooperation potential is:

$\coopPot(OrgA,OrgB,OrgC,S) = \{D, F, H\} - \{D, F\} $

$\coopPot(OrgA,OrgB,OrgC,S) = \{H\} $



In [5]:
# Run the cooperation potential with metage2metabo on the folder of SBML using the seeds files.
# This is similar to the command:
# m2m addedvalue -n data/community -s data/seeds.sbml -o output_folder
from metage2metabo.m2m import community_addedvalue, individual_scope, community_scope

networks_path = 'data/community/'
seeds_path = 'data/seeds.sbml'
output_folder = 'output_folder'

iscope_metabolites = individual_scope.iscope(networks_path, seeds_path, output_folder)
instance_comscope, cscope_metabolites = community_scope.cscope(networks_path, seeds_path, output_folder)

addedvalue = community_addedvalue.addedvalue(iscope_metabolites, cscope_metabolites, output_folder)

In [6]:
print('The cooperation potential contains {0} metabolites: {1}.'.format(len(addedvalue), ', '.join(addedvalue)))

The cooperation potential contains 1 metabolites: M_H_c.


## `m2m mincom`: Minimal community selection


### Minimal community formalism

A minimal community $\mathcal{C}$ enabling the producibility of a set of targets $T$ from the seeds $S$ is a sub-family of the community $G_1, \dots, G_n$ which is solution of the following optimisation problem:

$SetofMinimalCommunity(G_1 .. G_n, S, T) =  arg \, min ( size( {\{G_i..G_j\}} \, | \, \mathcal{P}_{\{G_1..G_n\}} \subset  \Sigma_{\{G_i..G_j\}}(S) )$

$$\begin{array}{lll}
 \qquad \qquad  & 
 \displaystyle{
 \minimize_{ \{G_{i_1}.. G_{i_L}\}  \  \subset \{G_{1}.. G_{N}\} }
 }
  & \mbox{size} (\{G_{i_1}.. G_{i_L}\})   \\ 
& \mbox{subject to }  &   T \subset  \mixedbagScope(G_{i_1} .. G_{i_L}),S). 
\end{array}       
$$

### Example

In this example, we have 8 species. In order to produce the targets from the seeds, there are 4 minimal communities of size of 3 organisms.

The yellow organism occurs in all minimal communities, it is an essential symbiont.
Green, orange, cyan and purple organisms occur in some minimal communities but not in all, they are alternatives symbionts.

<p>
    <img src='images/key_species.png' width='60%'>
    <center><em>Figue 3: Key species</em></center>
</p>


### Key species

Many minimal communities are expected to be equivalent for a given metabolic objective but their enumeration can be computationally costly.

We define *key species* which are organisms occurring in at least one community among all the optimal ones. Key species can be further distinguished into *essential symbionts* and *alternative symbionts*. The former occur in every minimal community whereas the latter occur only in some minimal communities. More precisely, the key species $\ks(G_1 .. G_n, S,T)$, the essential symbionts $\es(G_1 .. G_n, S,T)$, and the alternative symbionts $\as(G_1 .. G_n, S,T)$ associated to a set of metabolic networks, seeds $S$ and a set of target metabolites $T$ are defined by:


\begin{align*}
    \ks(G_1 .. G_n, S,T) &= \{ G \mid \exists {\mathcal C} \in \mincom(G_1 .. G_n, S, T), \, G \in {\mathcal C}\}. \\
    \es(G_1 .. G_n, S,T) &= \{ G \mid \forall {\mathcal C} \in \mincom(G_1 .. G_n, S, T), \, G \in {\mathcal C}\}. \\
    \as(G_1 .. G_n, S,T) &= \ks(G_1 .. G_n, S,T) \setminus \es(G_1 .. G_n, S,T).
\end{align*}

In [7]:
# Run the minimal community with metage2metabo on the folder of SBML using the seeds files.
# This is similar to the command:
#m2m metacom -n data/community -s data/seeds.sbml -o output_folder
from metage2metabo.m2m import community_addedvalue, individual_scope, community_scope
from metage2metabo import sbml_management

networks_path = 'data/community/'
seeds_path = 'data/seeds.sbml'
output_folder = 'output_folder'
target_path = 'targets.sbml'

iscope_metabolites = individual_scope.iscope(networks_path, seeds_path, output_folder)
instance_comscope, cscope_metabolites = community_scope.cscope(networks_path, seeds_path, output_folder)
addedvalue = community_addedvalue.addedvalue(iscope_metabolites, cscope_metabolites, output_folder)

if len(addedvalue):
    sbml_management.create_species_sbml(addedvalue, target_path)
instance_mincom = community_scope.instance_community(networks_path, seeds_path, output_folder, target_path)

## `m2m_analysis`: Visualisation of minimal communities

To visualize the results, we will use the m2m_analysis part of m2m.

In [8]:
from metage2metabo.m2m_analysis import m2m_analysis_workflow

m2m_analysis_workflow.run_analysis_workflow(networks_path, target_path, seeds_path, output_folder, None, None)

/home/abelcour/.local/lib/python3.8/site-packages/miscoto/encodings/community_soup.lp


Minimal communities:
- OrgA X OrgB
- OrgA X OrgC

First, m2m_analysis will compute the solution graph. This graph node corresponds to organism from the community. An edge is drawn between two nodes if the two organisms occur in the same minimal community.

In this example, m2m_analysis will draw an edge between OrgA and OrgB and another edge between OrgA and OrgC.

<p>
    <img src='images/graph_solution.png' width='40%'>
    <center><em>Figue 4: Solution graph</em></center>
</p>

Then m2m_analysis will compress the solution graph into a powergraph. To do this, m2m_analysis uses powergrasp, which will search for pattern (like hub or clique) and compress them into powernode and poweredge. Here, as node OrgA is a hub in the graph (as connected to the other nodes of the graph), a poweredge will be drawn between OrgA node and a powernode containing both OrgB and OrgC.

OrgA occurs in all the minimal community, it is an essential symbiont (in dark pink).

OrgB and OrgC occur in one minimal community but not all, they are alternative symbionts (in blue).

<p>
    <img src='images/targets_powergraph.png' width='40%'>
    <center><em>Figue 5: Powergraph</em></center>
</p>

# Tutorial on real data

For this tutorial, we used a subset of the data used in the article of M2M (avaible in the [Github repository](https://github.com/AuReMe/metage2metabo/tree/master/test/metabolic_data)).

- `toy_bact`: 17 metabolic networks
- `seeds_toy.sbml`: 93 metabolites (classical diet for the gut microbiota, EU average from the VMH resource and a small number of currency metabolites).



In [2]:
# Instanciate the logger
import logging
import statistics
import json
import os

logging.basicConfig()
logging.setLevel = logging.CRITICAL
# Run the individual scope with metage2metabo on the folder of SBML using the seeds files.
# This is similar to the command:
# m2m iscope -n data/community -s data/seeds.sbml -o output_folder
from metage2metabo.m2m import individual_scope

networks_path = 'toy_bact'
seeds_path = 'seeds_toy.sbml'
output_folder = 'tutorial_output_folder'
indiv_output = os.path.join(output_folder, 'indiv_scopes')
indiv_scope_json = os.path.join(indiv_output, 'indiv_scopes.json')
key_species_file = os.path.join(output_folder, 'key_species.json')
target_path = 'targets.sbml'

individual_scope.iscope(networks_path, seeds_path, output_folder)

# Read the result
with open(indiv_scope_json) as iscope_output:
    iscope_json_data = json.load(iscope_output)

metabolites = []
union_metabolites = set()
for species in iscope_json_data:
    metabolites.append(len(iscope_json_data[species]))
    union_metabolites.update(iscope_json_data[species])

print('Average individual scope: {0}'.format(statistics.mean(metabolites)))

# Run the community scope with metage2metabo on the folder of SBML using the seeds files.
# This is similar to the command: m2m cscope -n toy_bact -s seeds_toy.sbml -o output_folder
from metage2metabo.m2m import community_scope

instance_path, network_scopes = community_scope.cscope(networks_path, seeds_path, output_folder)

print('The community scope contains {0} metabolites.'.format(len(network_scopes)))

print('Cooperation potential: {0} metabolites.'.format(len(set(network_scopes)-union_metabolites)))
# Run the minimal community with metage2metabo on the folder of SBML using the seeds files.
# This is similar to the command: m2m metacom -n toy_bact -s seeds_toy.sbml -o output_folder
from metage2metabo.m2m import community_addedvalue, individual_scope, community_scope
from metage2metabo import sbml_management

iscope_metabolites = individual_scope.iscope(networks_path, seeds_path, output_folder)
instance_comscope, cscope_metabolites = community_scope.cscope(networks_path, seeds_path, output_folder)
addedvalue = community_addedvalue.addedvalue(iscope_metabolites, cscope_metabolites, output_folder)

if len(addedvalue):
    sbml_management.create_species_sbml(addedvalue, target_path)
instance_mincom = community_scope.instance_community(networks_path, seeds_path, output_folder, target_path)

from metage2metabo.m2m_analysis import m2m_analysis_workflow

m2m_analysis_workflow.run_analysis_workflow(networks_path, target_path, seeds_path, output_folder, None, None)

with open(key_species_file) as json_key_species:
    key_species_json_data = json.load(json_key_species)
    
nb_essential_symbionts = len(key_species_json_data['targets']['essential_symbionts']['data'])
essential_symbionts = ','.join(key_species_json_data['targets']['essential_symbionts']['data'])
nb_alternative_symbionts = len(key_species_json_data['targets']['alternative_symbionts']['data'])
alternative_symbiotns = ','.join(key_species_json_data['targets']['alternative_symbionts']['data'])
print('{0} Essential symbionts: {1}'.format(nb_essential_symbionts, essential_symbionts))
print('{0} Alternative symbionts: {1}'.format(nb_alternative_symbionts, alternative_symbiotns))

Average individual scope: 308.70588235294116
The community scope contains 651 metabolites.
Cooperation potential: 119 metabolites.
/mnt/c/Users/Arnaud/Downloads/Work_directory/programs/miscoto/miscoto/encodings/community_soup.lp
12 Essential symbionts: GCA_003437055,GCA_003437905,GCA_003437815,GCA_003437885,GCA_003437715,GCA_003437295,GCA_003437195,GCA_003437665,GCA_003437595,GCA_003437255,GCA_003438055,GCA_003437375
5 Alternative symbionts: GCA_003437325,GCA_003437175,GCA_003437785,GCA_003437345,GCA_003437945


If you want more information on the output files:

- result files of m2m are described [here](https://metage2metabo.readthedocs.io/en/latest/output.html).
- result files of m2m_analysis are explained [here](https://metage2metabo.readthedocs.io/en/latest/m2m_analysis.html#m2m-analysis-output-files).